# Программирование на Python
## Семинар 5. Повторение основ

Имеются данные по сотрудникам некоторой Компании. Они устроены следующим образом. Компания подразделяется на Блоки, Блоки - на Департаменты. Название Департамента необязательно уникально. От отдела кадров по результатам некоторого тестирования вам пришли файлы со статистиками по сотрудникам. Они находятся в папке `data`.

Также существует справочник электронных почт руководителей Департаментов и шаблон письма (папка `sources`).

#### Задача

- изучить файлы от отдела кадров, понять общую структуру;
- на их основании определить среднее значение по каждой из статистик в разрезе Департамента (и проинтерпретировать его - выше или ниже среднего значения по всей компании);
- составить по результатам письмо руководителю каждого из Департаментов;
- автоматически разослать его, используя ваш корпоративный аккаунт.

In [1]:
from pathlib import Path  # для ООП-работы с файловой системой
import openpyxl
from email.mime.text import MIMEText  # для создание email'а по протоколу MIME (для дальнейшей передачи)
from email.mime.multipart import MIMEMultipart
import smtplib  # для SMTP подключения
from configparser import ConfigParser  # для парсинга конфигов
from time import sleep  # для задержки при отправке

In [ ]:
# PLAN
## parse_info; input: path, output: dict[(block, department), ] +++++
## parse_data; input: path, output: dict[(block, department), ], with length of 1 +++++
## iterate through files, extract tables, save them: dict[(block, department), ] +++++
## collect values for value1/2 together, compute +++++
## iterate through keys, fill template with values -> text
## create email
## send email

In [2]:
# consts.py
DATA_PATH_STR = 'data'
SOURCE_PATH_STR = 'sources'
INFO_NAME = 'Справочник.xlsx'
TEMP_NAME = 'message_template.html'
CONF_NAME = 'config.ini'
COLNAMES_MAPPER = {
    'ID': 'id',
    'Показатель 1': 'value1',
    'Показатель 2': 'value2',
}
UNKNOWN_COL_VALUE = 'UNKNOWN_COL'
INTERP_MAPPER = {
    True: 'значение НИЖЕ среднего по всей компании',
    False: 'значение не ниже среднего по всей компании',
}
NDIGITS = 2

In [3]:
# functions.py
def parse_info(path: Path) -> dict:
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    info_dict = {
        (row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value}
        for row
        in ws.iter_rows()
        if row[0].value != 'Блок'
    }

    wb.close()

    return info_dict

def parse_table(path: Path) -> dict:
    department_name = path.name.rstrip('.xlsx')
    block = path.parent.name
    
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    department = {}

    for col in ws.iter_cols():
        values = [cell.value for cell in col if cell.value is not None]
        colname = COLNAMES_MAPPER.get(values[0], UNKNOWN_COL_VALUE)
        colvalues = values[1:]
    
        department.update({colname: colvalues})

    wb.close()

    return {(block, department_name): department}

def mean(vector) -> float:
    return sum(vector) / len(vector)

In [4]:
# main.py
data_path = Path(DATA_PATH_STR)
source_path = Path(SOURCE_PATH_STR)

info_path = source_path / INFO_NAME
template_path = source_path / TEMP_NAME
config_path = source_path / CONF_NAME
filenames = data_path.glob('Блок */Департамент *.xlsx')

# parse configs
cp = ConfigParser()
_ = cp.read(config_path)
smtp_config = cp['smtp.client']

# parse template
with open(template_path, 'r') as file:
    template = file.read()

# parse info
info = parse_info(info_path)

# parse tables and merge them
datas = {}

value1 = []
value2 = []

for path in filenames:
    data = parse_table(path)
    datas.update(data)

    val = list(data.values())[0]

    value1.extend(val['value1'])
    value2.extend(val['value2'])

value1_mean = mean(value1)
value2_mean = mean(value2)

# create server object & authenticate
server = smtplib.SMTP_SSL(smtp_config['host'], smtp_config['port'])
server.login(smtp_config['login'], smtp_config['password'])

# create messages & send them
for key, value in info.items():
    data = datas.get(key)
    
    if data is None:
        continue
    
    format_dict = {}

    # update with info
    block, department = key
    
    format_dict.update({'fio': value['fio'], 'block': block, 'department': department})
    
    # update with data
    value1_mean_ = mean(data['value1'])
    value2_mean_ = mean(data['value2'])

    interp1 = INTERP_MAPPER[value1_mean_ < value1_mean]
    interp2 = INTERP_MAPPER[value2_mean_ < value2_mean]

    format_dict.update(
        {
            'value1': round(value1_mean_, NDIGITS),
            'value2': round(value2_mean_, NDIGITS),
            'interp1': interp1,
            'interp2': interp2
        }
    )

    message_text = template.format(**format_dict)

    # create email
    email = MIMEMultipart()  # create multipart email (text will be one oh the parts)

    email['From'] = smtp_config['login']
    email['To'] = value['email']

    subject = f'Отчет по {block}, {department}'
    email['Subject'] = subject
    
    mimetext = MIMEText(message_text, 'html')

    email.attach(mimetext)  # attach text to multipart email

    # send email
    send_status = server.sendmail(email['From'], email['To'], email.as_string())
    if send_status == {}:
        message_to_user = f'Email for {block}, {department} successfully sent!'
    else:
        message_to_user = f'For {block}, {department} something went wrong:('

    print(message_to_user)

    # wait for a second until next sending
    sleep(1)

server.close()

In [7]:
datas.get(('Блок 1', 'Департамент A'))

{'id': ['f660317c-6223-41fb-b131-b2b34e10767c',
  '2e8b6ace-9383-49dd-8214-d76c6b9649f2',
  'd855ad41-e5fa-4f85-80f8-07e037523597',
  'e5329fc2-7c32-4caa-8aad-0cdfd63c26a7',
  '6137cf2d-23ea-4413-bb1e-d24351fc28f0',
  '5c745038-6192-4eb6-834d-8ad54393229f',
  '57135763-fa5c-41b7-bff9-1e4d22ef91a1',
  'bfd4cb5e-a4dd-44bd-9eb0-966e8600d1d6',
  'bae1c82e-c47e-460f-88bf-220b64c78b3d',
  '31fa27df-e77c-47f7-b8e1-f2807be0a31f',
  'c51f8ba2-c138-4361-9db7-1f8b47b837c9',
  'e359a63d-ed95-4a3d-acc2-316c6e14f7d8',
  '21ef00f0-766f-468e-b163-50dcc8537a02',
  '2b218c55-1496-48d0-80d6-14f63a37eac8',
  '20e2beb0-3a57-47b0-adee-2a4586f1b167',
  '29975235-8164-4a7c-a46a-9e456b270ddd',
  '6964ae31-4eb5-423a-91cb-e2723ad603e5',
  'a61fbdc9-e17a-468c-8ac7-c28fd6aa5db9',
  '3f16c43f-43fb-4b6c-9919-132a327f2597',
  'a76d6e30-4ab5-4943-ba10-e0291c3bea30',
  '4cb63052-99ae-4175-b360-fbdef69262f7',
  '165b1c17-bcd8-47ca-bfcf-710f91c53e6f',
  '3cc0d328-7e6c-49c6-ae6c-da276fe956e0',
  'c0347f2e-eb2e-4ae6-8c71-0

In [9]:
wb = openpyxl.load_workbook(info_path)
ws = wb.active

In [19]:
example_filename = [*data_path.glob('Блок */Департамент *.xlsx')][0]

In [20]:
wb = openpyxl.load_workbook(example_filename)
ws = wb.active

In [31]:
example_filename.name.rstrip('.xlsx')

'Департамент A'

In [33]:
example_filename.parent.name

'Блок 3'

In [26]:
department = {}

for col in ws.iter_cols():
    values = [cell.value for cell in col if cell.value is not None]
    colname = COLNAMES_MAPPER.get(values[0], UNKNOWN_COL_VALUE)
    colvalues = values[1:]

    department.update({colname: colvalues})

In [15]:
info_dict = {
    (row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value}
    for row
    in ws.iter_rows()
    if row[0].value != 'Блок'
}

{('Блок 1', 'Департамент A'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 1', 'Департамент B'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 1', 'Департамент C'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент A'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент B'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент C'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент D'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент E'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент F'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 

In [28]:
wb.close()

In [5]:
data_path_str = 'data'
data_path = Path(data_path_str)

In [10]:
paths = [*data_path.glob('Блок */Департамент *.xlsx')]

In [11]:
paths

[PosixPath('data/Блок 3/Департамент A.xlsx'),
 PosixPath('data/Блок 3/Департамент B.xlsx'),
 PosixPath('data/Блок 4/Департамент A.xlsx'),
 PosixPath('data/Блок 5/Департамент A.xlsx'),
 PosixPath('data/Блок 5/Департамент L.xlsx'),
 PosixPath('data/Блок 5/Департамент G.xlsx'),
 PosixPath('data/Блок 5/Департамент K.xlsx'),
 PosixPath('data/Блок 5/Департамент J.xlsx'),
 PosixPath('data/Блок 5/Департамент F.xlsx'),
 PosixPath('data/Блок 5/Департамент E.xlsx'),
 PosixPath('data/Блок 5/Департамент I.xlsx'),
 PosixPath('data/Блок 5/Департамент H.xlsx'),
 PosixPath('data/Блок 5/Департамент D.xlsx'),
 PosixPath('data/Блок 5/Департамент C.xlsx'),
 PosixPath('data/Блок 5/Департамент B.xlsx'),
 PosixPath('data/Блок 2/Департамент A.xlsx'),
 PosixPath('data/Блок 2/Департамент G.xlsx'),
 PosixPath('data/Блок 2/Департамент F.xlsx'),
 PosixPath('data/Блок 2/Департамент E.xlsx'),
 PosixPath('data/Блок 2/Департамент D.xlsx'),
 PosixPath('data/Блок 2/Департамент C.xlsx'),
 PosixPath('data/Блок 2/Департамен